# Analyze Dataset

In [ ]:
import glob
import json

import pandas as pd

dataset = pd.read_parquet('../data/raw/lfm_popularity/dataset_20.parquet')
dataset_cols = list(filter(lambda x: not x.startswith('highlevel') and not x.startswith('lowlevel') and not x.startswith('rhythm') and not x.startswith('tonal'), list(dataset.columns)))

charts = pd.read_csv('/storage/nas3/datasets/music/billboard/billboard_hot-100_1958-08-11_2019-07-06.csv', header=0, index_col=0)
visited = pd.read_csv('../data/interim/lfm_popularity/mp3s/visited.csv', header=0, index_col=0)

charts_dataset = dataset.merge(charts, left_on=['artist_name', 'recording_name'], right_on=['artist', 'title'])
visited_dataset = dataset.merge(visited, left_on=['recording_mbid'], right_on=['track_id'])

visited_charts_dataset = visited_dataset.merge(charts_dataset, on=['recording_mbid'])

## Show duplicates

In [ ]:
duplicates = dataset[dataset.duplicated(['artist_name', 'recording_name'], keep=False)]
duplicates.sort_values(['artist_name', 'recording_name'], inplace=True)
duplicates = duplicates[duplicates['listener_count'] > 100_000]
display(duplicates[dataset_cols])

In [ ]:
## Charts

In [ ]:
charts_dataset_cols = dataset_cols + ['lastPos', 'peakPos', 'rank', 'weeks', 'latest-date']

display(charts_dataset[charts_dataset_cols])

In [ ]:
visited_charts_dataset = visited_charts_dataset.merge(dataset, on=['recording_mbid'])
visited_charts_dataset = visited_charts_dataset.merge(yt_info, on=['recording_mbid'])


In [ ]:
import seaborn as sns

corr = charts_dataset[['listener_count','play_count','peakPos', 'rank', 'weeks']].corr()
sns.heatmap(corr, annot=True).set_title('Pearson')

In [ ]:
kendal_corr = charts_dataset[['listener_count','play_count','peakPos', 'rank', 'weeks']].corr(method='kendall')
sns.heatmap(kendal_corr, annot=True).set_title('kendall')

In [ ]:
# visited_yt_info = []
# files = glob.glob('../data/interim/lfm_popularity/mp3s/*.info.json')
# display(len(files))
# count = 0
# for _, row in visited_dataset.iterrows():
#     file = list(filter(lambda f: row['track_id'] in f, files))
#     try:
#         with open(file[0]) as json_file:
#             info = json.load(json_file)
#             info['recording_mbid'] = row['track_id']
#             visited_yt_info.append(info)
#     except:
#         continue
        
#     count += 1
    
#     if count > 100:
#         break

# yt_info = pd.DataFrame(visited_yt_info)    
# display(yt_info)